In [ ]:
!pip install "pyiceberg[s3fs,hive]"

In [46]:
from pyiceberg.catalog import load_catalog
catalog = load_catalog(
        "docs",
    **{
            "uri": "thrift://hive-ms-trino.apps.zagaopenshift.zagaopensource.com:9888",
            "s3.endpoint": "http://minio-lb.apps.zagaopenshift.zagaopensource.com:9009",
            "py-io-impl": "pyiceberg.io.pyarrow.PyArrowFileIO",
            "s3.access-key-id": "minioAdmin",
            "s3.secret-access-key": "minio1234",
    }
)

In [48]:
namespace="docs_logs" #enter namespace
ns=catalog.list_namespaces()
print(ns)
if((namespace,) in ns):
  print("Namespace alredy exists")
else:
  catalog.create_namespace(namespace)
  print("Namespace create suceesss")



[('data',), ('default',), ('demo_data',), ('demo_saro',), ('demo_trace',), ('docs_check',), ('docs_logs',), ('docs_mari',), ('docs_powerleveldata',), ('docs_sa',), ('docs_sar',), ('docs_saro',), ('docs_selva',), ('hi',), ('hlo',), ('ingesting',), ('new',), ('old',), ('poc',), ('pod',), ('pod_data',), ('power_pod',), ('powerlevelpoddata',), ('powerpod',), ('saravan',), ('saravana',), ('saro',), ('saro_data',), ('sh',), ('sharanya',), ('sk',), ('soh',), ('trace_ingestion_data',)]
Namespace alredy exists


In [ ]:
# catalog.drop_namespace("docs_logs")

In [49]:
from pyiceberg.schema import Schema
from pyiceberg.types import (
    TimestampType,
    LongType,
    IntegerType,
    FloatType,
    DoubleType,
    StringType,
    NestedField,
    StructType,
    ListType
)
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.transforms import DayTransform
from pyiceberg.table.sorting import SortOrder, SortField
from pyiceberg.transforms import IdentityTransform

# Define the schema for the spans
schema = Schema(
    NestedField(field_id=1, name="servicename", field_type=StringType(), required=False),
    NestedField(field_id=2, name="language", field_type=StringType(), required=False),
    NestedField(field_id=3, name="logsdata", field_type=StringType(), required=False),
    NestedField(field_id=4, name="createdTime", field_type=TimestampType(), required=False),

)

# partition_spec = PartitionSpec(
#     PartitionField(
#         source_id=1, field_id=1000, transform=DayTransform(), name="datetime_day"
#     )
# )

sort_order = SortOrder(SortField(source_id=3, transform=IdentityTransform()))

table_name = "logs"  # Define your table identifier


table = catalog.create_table(
    identifier=(namespace, table_name),
    schema=schema,
    location="s3a://apmlogs/logs", #mention s3 bucketname
    # partition_spec=partition_spec,
    sort_order=sort_order,
)
print(f"Table '{table_name}' created successfully.")



Table 'logs' created successfully.


In [54]:
table.schema()

Schema(NestedField(field_id=1, name='servicename', field_type=StringType(), required=False), NestedField(field_id=2, name='language', field_type=StringType(), required=False), NestedField(field_id=3, name='logsdata', field_type=StringType(), required=False), NestedField(field_id=4, name='createdTime', field_type=TimestampType(), required=False), schema_id=0, identifier_field_ids=[])

In [55]:
catalog.list_tables("docs_logs")  #mention namespace

[('docs_logs', 'logs')]

In [44]:
#catalog.drop_table("docs_logs.logs") #mention namespace and tablename